<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/TrivagoFeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

## Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [0]:
# !pip install cudf-cuda100
# !cp /usr/local/lib/python3.6/dist-packages/librmm.so .
# import os  
# os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
# os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

In [0]:
# import cudf 
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
from datetime import datetime
import re
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
item_metadata_filepath = '../../Datasets/raw_data/item_metadata.csv'
item_metadata = pd.read_csv(item_metadata_filepath)

train_filepath = '../../Datasets/clean_data/train.csv'
train = pd.read_csv(train_filepath)

## Important Dataframes Preparation

In engineer global features from the train dataset, which will be most of the cases around either clickouts or final click out examples (instances), two dataframes of them will be created.

In order to get representative global values to the items, duplications must be removed. Rows would not all be duplicated because of some attributes such as timestamp, reference, and step.

In [0]:
train.drop('Unnamed: 0', axis=1, inplace=True)
ClickoutDF = train[train.action_type=='clickout item']
FinalClickoutDF = train[train.action_type=='clickout item'].groupby('session_id').tail(1)
ClickoutUniqueDF = ClickoutDF.drop_duplicates(subset=['session_id', 'impressions'], keep='first')

## Item Global Features
### Number of Properties

In [7]:
item_metadata.properties = item_metadata.properties.apply(lambda x: x.split('|'))
item_metadata['NumberOfProperties'] = item_metadata.properties.apply(lambda x: len(x))
item_metadata.head(2)

,item_id,properties,NumberOfProperties
0,5101,"[Satellite TV, Golf Course, Airport Shuttle, C...",62
1,5416,"[Satellite TV, Cosmetic Mirror, Safe (Hotel), ...",46


In [8]:
#getting total number of unique properties across all items
AllPropertiesList = item_metadata.properties.tolist()

AllPropertiesFlatList = []
for sublist in AllPropertiesList:
    for item in sublist:
        AllPropertiesFlatList.append(item)
        
print('Number of unique properties is', len(set(AllPropertiesFlatList)))

Number of unique properties is 157


### Items Properties Similarities
The purpose is to get cosine similarity between items. The maximum 25 items out of the list can be extracted as a dataframe and cosine similarity can be obtained in order to get the similar items to the ones the user had interacted with to be listed on the top of the list.  

In [9]:
item_metadata.properties = item_metadata.properties.apply(lambda x: tuple(x))

one_hot = MultiLabelBinarizer()

properties_encoded = one_hot.fit_transform(item_metadata.properties.values.tolist())

properties_encodedDF = pd.DataFrame(properties_encoded)

#changing column names
properties_list = one_hot.classes_.tolist()
for i in range(len(properties_list)):
    properties_encodedDF = properties_encodedDF.rename(columns={i:properties_list[i]})

#creating a column of the item id to get the similarity between items
item_metadata.item_id = item_metadata.item_id.apply(lambda x: str(x))
properties_encodedDF['item_id'] = item_metadata.item_id

properties_encodedDF.head()

,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,Airport Shuttle,All Inclusive (Upon Inquiry),Balcony,Bathtub,Beach,Beach Bar,Beauty Salon,Bed & Breakfast,Bike Rental,Boat Rental,Body Treatments,Boutique Hotel,Bowling,Bungalows,Business Centre,Business Hotel,Cable TV,Camping Site,Car Park,Casa Rural (ES),Casino (Hotel),Central Heating,Childcare,Club Hotel,Computer with Internet,Concierge,Conference Rooms,Convenience Store,Convention Hotel,Cosmetic Mirror,...,Satellite TV,Satisfactory Rating,Sauna,Self Catering,Senior Travellers,Serviced Apartment,Shooting Sports,Shower,Singles,Sitting Area (Rooms),Ski Resort,Skiing,Solarium,Spa (Wellness Facility),Spa Hotel,Steam Room,Sun Umbrellas,Surfing,Swimming Pool (Bar),Swimming Pool (Combined Filter),Swimming Pool (Indoor),Swimming Pool (Outdoor),Szep Kartya,Table Tennis,Telephone,Teleprinter,Television,Tennis Court,Tennis Court (Indoor),Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms),item_id
0,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,1,0,1,0,0,1,...,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,5101
1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,...,1,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,1,5416
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,...,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,5834
3,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,1,1,0,0,1,...,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,5910
4,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,0,1,0,0,1,0,1,0,1,1,...,1,1,1,0,0,0,0,1,0,1,0,0,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1,6066


#### K-means

#### PCA

### Number of Times in Impressions
The purpose of this feature is to check how many times an item had been shown to users in the list.

Finding number of times an item has been mentioned in a list.

In [10]:
AllImpressionsList = ClickoutUniqueDF.impressions.apply(lambda x:x.split('|'))

AllImpressionsFlatList = []
for sublist in AllImpressionsList:
    for item in sublist:
        AllImpressionsFlatList.append(item)

InImpressionsCounter = Counter(AllImpressionsFlatList)
InImpressionsDF = pd.DataFrame.from_dict(InImpressionsCounter, orient='index').reset_index()\
                              .rename(columns={'index':'item_id', 0:'NumberInImpressions'})
InImpressionsDF.head(2)

,item_id,NumberInImpressions
0,3400638,491
1,1253714,84


In [11]:
InImpressionsDF.item_id.nunique(), len(InImpressionsDF), item_metadata.item_id.nunique()

(815092, 815092, 927142)

Number of items in this dataframe is less than the number of items in item_metadata, that's because some of the items had not been mentioned in the impressions list.

In [12]:
item_metadata.item_id = item_metadata.item_id.apply(lambda x: str(x))
item_metadata.drop(columns='properties', inplace=True)

#left joining
item_metadata = item_metadata.merge(InImpressionsDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions
0,5101,62,89.0
1,5416,46,63.0


### Number of Times in Reference
The purpose of this feature is to check how many times an item has been mentioned in the Reference attribute in the whole train set. 

In [13]:
InReferencesCounter = Counter(train.reference.values.tolist())
InReferencesDF = pd.DataFrame.from_dict(InReferencesCounter, orient='index').reset_index()\
                              .rename(columns={'index':'item_id', 0:'NumberInReferences'})

InReferencesDF.head(2)

,item_id,NumberInReferences
0,Newtown,16
1,666856,26


In [14]:
#left joining
item_metadata = item_metadata.merge(InReferencesDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences
0,5101,62,89.0,14.0
1,5416,46,63.0,43.0


### Number of Times in Clickout
The purpose of this feature is to check how many times an item has been clicked out

In [15]:
InClickoutCounter = Counter(ClickoutDF.reference.values.tolist())
InClickoutDF = pd.DataFrame.from_dict(InClickoutCounter, orient='index').reset_index()\
                              .rename(columns={'index':'item_id', 0:'NumberAsClickout'})
InClickoutDF.head(2)

,item_id,NumberAsClickout
0,109038,53
1,1257342,20


In [16]:
#left joining
item_metadata = item_metadata.merge(InClickoutDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout
0,5101,62,89.0,14.0,7.0
1,5416,46,63.0,43.0,6.0


### Number of Time in Final Clickout
The purpose of this feature is to get the number of times an item had been mentioned as a final clickout.

In [17]:
InFinalClickoutCounter = Counter(FinalClickoutDF.reference.values.tolist())
InFinalClickoutDF = pd.DataFrame.from_dict(InFinalClickoutCounter, orient='index').reset_index()\
                              .rename(columns={'index':'item_id', 0:'NumberAsFinalClickout'})
InFinalClickoutDF.head(2)

,item_id,NumberAsFinalClickout
0,1257342,7
1,2795374,18


In [18]:
#left joining
item_metadata = item_metadata.merge(InFinalClickoutDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout
0,5101,62,89.0,14.0,7.0,4.0
1,5416,46,63.0,43.0,6.0,2.0


The next step would be dividing the NumberAsFinalClickout by the other 3, in order to get FinalClickout Relativity

### Final Clickout To Impressions
The purpose of this feature is to get item's rate of clicking out when listed to the user.

In [19]:
FClickoutToImpressions = item_metadata.NumberAsFinalClickout/item_metadata.NumberInImpressions
FClickoutToImpressions.head(2)

0    0.044944
1    0.031746
dtype: float64

In [20]:
#adding attribute
item_metadata['FClickoutToImpressions'] = FClickoutToImpressions
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions
0,5101,62,89.0,14.0,7.0,4.0,0.044944
1,5416,46,63.0,43.0,6.0,2.0,0.031746


### Final Clickout To References
The purpose of this feature is to get item's rate of clicking out when it was interacted with.

In [21]:
FClickoutToReferences = item_metadata.NumberAsFinalClickout/item_metadata.NumberInReferences
FClickoutToReferences.head(2)

0    0.285714
1    0.046512
dtype: float64

In [22]:
#adding attribute
item_metadata['FClickoutToReferences'] = FClickoutToReferences
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512


### Final Clickout To Clickout
The purpose of this feature is to get item's rate of clickout when it was clicked out before.

In [23]:
FClickoutToClickout = item_metadata.NumberAsFinalClickout/item_metadata.NumberAsClickout
FClickoutToClickout.head(2)

0    0.571429
1    0.333333
dtype: float64

In [24]:
#adding attribute
item_metadata['FClickoutToClickout'] = FClickoutToClickout
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333


### Item's Average Rank
The purpose of this feature is to get item's position in the list provided to the user across the train set.  
Since the purpose is to get the average rank across the lists shown to users, an important notice which is that some sessions have different reference and click outs, which provides the same impression list. Duplicated impressions lists in each session should be dropped. (Same thing applies to price as well.)

In [25]:
# using All Clickout dataframe, but the one with the unique impressions for each session for this feature
ClickoutUniqueDF.head(1)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...


In [26]:
SessionImpressionsDF = ClickoutUniqueDF[['session_id', 'impressions']]
SessionImpressionsDF.impressions = SessionImpressionsDF.impressions.apply(lambda x: x.split('|'))
SessionImpressionsDF.impressions.head(2)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


13    [3400638, 1253714, 3367857, 5100540, 1088584, ...
15    [55109, 129343, 54824, 2297972, 109014, 125734...
Name: impressions, dtype: object

In [27]:
SessionImpressionsDF_exploded = SessionImpressionsDF.explode('impressions')
SessionImpressionsDF_exploded = SessionImpressionsDF_exploded.rename(columns={'impressions':'item_id'})
SessionImpressionsDF_exploded.head(2)

,session_id,item_id
13,aff3928535f48,3400638
13,aff3928535f48,1253714


In [28]:
#creating a rank column
rank = SessionImpressionsDF.impressions.apply(lambda x: list(range(1, (len(x) + 1))))
SessionRankDF = pd.concat([SessionImpressionsDF.session_id, rank], axis=1)
SessionRankDF = SessionRankDF.rename(columns={'impressions':'rank'})
SessionRankDF_exploded = SessionRankDF.explode('rank')
SessionRankDF_exploded.head(2)

,session_id,rank
13,aff3928535f48,1
13,aff3928535f48,2


In [29]:
#creating dataframe that combines both items and rank
ItemRankDF = pd.DataFrame({'item_id':SessionImpressionsDF_exploded['item_id'].values.tolist(),
                            'rank':SessionRankDF_exploded['rank'].values.tolist()})
ItemRankDF.head(2)

,item_id,rank
0,3400638,1
1,1253714,2


In [30]:
#getting the mean of ranks for each item
ItemAverageRank = ItemRankDF.groupby('item_id', sort=False)['rank'].mean().to_frame().reset_index()
ItemAverageRank = ItemAverageRank.rename(columns={'rank':'MeanRank'})
ItemAverageRank.head(2)

,item_id,MeanRank
0,3400638,15.627291
1,1253714,14.321429


In [31]:
#left joining
item_metadata = item_metadata.merge(ItemAverageRank, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111


### Item's Average Price
The purpose of this feature is to get item's average price accross the train set.  
Making sure that the length of each impressions list is as the same length of the prices list.

In [32]:
ImpressionsLength = ClickoutUniqueDF.impressions.apply(lambda x: x.split('|')).apply(lambda x: len(x))
PricesLength = ClickoutUniqueDF.prices.apply(lambda x: x.split('|')).apply(lambda x: len(x))
ImpressionsLength.equals(PricesLength)

True

In [33]:
SessionPricesDF = ClickoutUniqueDF[['session_id', 'prices']]
SessionPricesDF.prices = SessionPricesDF.prices.apply(lambda x: x.split('|'))
SessionPricesDF_exploded = SessionPricesDF.explode('prices')
ItemPriceDF = pd.DataFrame({'item_id':SessionImpressionsDF_exploded.item_id.values.tolist(),
                            'price':SessionPricesDF_exploded.prices.values.tolist()})
ItemPriceDF.head(2)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,item_id,price
0,3400638,95
1,1253714,66


In [34]:
ItemPriceDF.price = ItemPriceDF.price.apply(lambda x: int(x))
ItemAveragePriceDF = ItemPriceDF.groupby('item_id', sort=False)['price'].mean().to_frame().reset_index()
ItemAveragePriceDF = ItemAveragePriceDF.rename(columns={'price':'MeanPrice'})
ItemAveragePriceDF.head(2)

,item_id,MeanPrice
0,3400638,192.154786
1,1253714,82.678571


In [35]:
item_metadata = item_metadata.merge(ItemAveragePriceDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539,121.696629
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111,102.873016


### Item's Maximum Price

In [36]:
ItemMaximumPriceDF = ItemPriceDF.groupby('item_id', sort=False)['price'].max().to_frame().reset_index()
ItemMaximumPriceDF = ItemMaximumPriceDF.rename(columns={'price':'MaxPrice'})
ItemMaximumPriceDF.head(2)

,item_id,MaxPrice
0,3400638,2523
1,1253714,248


In [37]:
#left joining
item_metadata = item_metadata.merge(ItemMaximumPriceDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice,MaxPrice
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539,121.696629,241.0
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111,102.873016,270.0


### Item's Minimum Price

In [38]:
ItemMinimumPriceDF = ItemPriceDF.groupby('item_id', sort=False)['price'].min().to_frame().reset_index()
ItemMinimumPriceDF = ItemMinimumPriceDF.rename(columns={'price':'MinPrice'})
ItemMinimumPriceDF.head(2)

,item_id,MinPrice
0,3400638,68
1,1253714,50


In [39]:
#left joining
item_metadata = item_metadata.merge(ItemMinimumPriceDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice,MaxPrice,MinPrice
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539,121.696629,241.0,64.0
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111,102.873016,270.0,59.0


### Price Rank
The purpose of this feature is to get the average price rank of an item across the train set. Sorting the items not by what is being showed to the user, but by the items' prices, getting the average price rank for each item across the whole train. It shows where an item stands from its peers.  
By using the argsort function, the price rank can be feature can be engineered.

In [40]:
ClickoutDF.prices[13]

'95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165'

In [41]:
np.argsort(list(map(int, ClickoutDF.prices[13].split('|'))))

array([13, 23,  9, 10,  1, 12, 14,  7, 16, 20,  8,  0,  4,  5,  6,  3, 19,
       21, 17, 22, 11, 15, 24, 18,  2])

In [42]:
SessionPricesDF['PriceRank'] = SessionPricesDF.prices.apply(lambda x: (list(map(int, x))))\
                                                     .apply(lambda x: np.argsort(x))
SessionPricesDF.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,session_id,prices,PriceRank
13,aff3928535f48,"[95, 66, 501, 112, 95, 100, 101, 72, 82, 56, 5...","[13, 23, 9, 10, 1, 12, 14, 7, 16, 20, 8, 0, 4,..."
15,aff3928535f48,"[162, 25, 150, 143, 101, 49, 118, 131, 18, 100...","[8, 1, 15, 5, 12, 16, 20, 9, 4, 10, 23, 24, 6,..."


In [43]:
SessionPricesRankDF = SessionPricesDF[['session_id', 'PriceRank']].explode('PriceRank')
SessionPricesRankDF.head(2)

,session_id,PriceRank
13,aff3928535f48,13
13,aff3928535f48,23


In [44]:
ItemPriceRankDF = pd.DataFrame({'item_id':SessionImpressionsDF_exploded.item_id.values.tolist(),
                            'PriceRank':SessionPricesRankDF.PriceRank.values.tolist()})
ItemPriceRankDF.head(2)

,item_id,PriceRank
0,3400638,13
1,1253714,23


In [45]:
#getting the average price rank of each item
ItemAveragePriceRankDF = ItemPriceRankDF.groupby('item_id', sort=False).PriceRank.mean().to_frame().reset_index()
ItemAveragePriceRankDF = ItemAveragePriceRankDF.rename(columns={'PriceRank':'AveragePriceRank'})
ItemAveragePriceRankDF.head(2)

,item_id,AveragePriceRank
0,3400638,11.826884
1,1253714,11.309524


In [46]:
# left joining
item_metadata = item_metadata.merge(ItemAveragePriceRankDF, on='item_id', how='left')
item_metadata.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice,MaxPrice,MinPrice,AveragePriceRank
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539,121.696629,241.0,64.0,12.303371
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111,102.873016,270.0,59.0,11.095238


In [47]:
#filling NaN values with zeros
item_metadata = item_metadata.fillna(0)

GlobalItemFeatures = item_metadata.merge(properties_encodedDF, on='item_id', how='left')
GlobalItemFeatures.head(2)

,item_id,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice,MaxPrice,MinPrice,AveragePriceRank,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,Airport Shuttle,All Inclusive (Upon Inquiry),Balcony,Bathtub,Beach,Beach Bar,Beauty Salon,Bed & Breakfast,Bike Rental,Boat Rental,Body Treatments,Boutique Hotel,Bowling,Bungalows,Business Centre,Business Hotel,...,Sailing,Satellite TV,Satisfactory Rating,Sauna,Self Catering,Senior Travellers,Serviced Apartment,Shooting Sports,Shower,Singles,Sitting Area (Rooms),Ski Resort,Skiing,Solarium,Spa (Wellness Facility),Spa Hotel,Steam Room,Sun Umbrellas,Surfing,Swimming Pool (Bar),Swimming Pool (Combined Filter),Swimming Pool (Indoor),Swimming Pool (Outdoor),Szep Kartya,Table Tennis,Telephone,Teleprinter,Television,Tennis Court,Tennis Court (Indoor),Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
0,5101,62,89.0,14.0,7.0,4.0,0.044944,0.285714,0.571429,12.168539,121.696629,241.0,64.0,12.303371,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,...,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1
1,5416,46,63.0,43.0,6.0,2.0,0.031746,0.046512,0.333333,10.111111,102.873016,270.0,59.0,11.095238,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,1


In [0]:
GlobalItemFeatures.to_csv('GlobalFeatures')

## Item Local Features

### General Features

In [7]:
session_gb = train.groupby('session_id', sort=False)
train.session_id.nunique()

745755

#### item_id

In [8]:
session = session_gb.get_group('aff3928535f48')
session_exploded = session[session.action_type=='clickout item'].tail(1)[['impressions']]
session_exploded.impressions = session_exploded.impressions.apply(lambda x: x.split('|'))
session_exploded = session_exploded.explode('impressions')
session_exploded = session_exploded.rename(columns={'impressions':'item_id'})
session_exploded.head(2)

,item_id
15,55109
15,129343


#### price

In [9]:
session_explode = session[session.action_type=='clickout item'].tail(1)[['prices']]
session_explode.prices = session_explode.prices.apply(lambda x: x.split('|'))
session_explode = session_explode.explode('prices')
session_explode = session_explode.rename(columns={'prices':'price'})
session_explode.price = session_explode.price.apply(lambda x: int(x))
session_explode.head(2)

,price
15,162
15,25


#### rank

In [0]:
session_exploded['rank'] = range(1, len(session_exploded) + 1)
session_exploded.head(2)

#### price_rank

In [10]:
session_exploded['price'] = session_explode.price
session_exploded['price_rank'] = np.argsort(session_exploded['price'])
session_exploded.head(2)

,item_id,price,rank,price_rank
15,55109,162,1,8
15,129343,25,2,1


#### clickout

In [11]:
clickout_item_id = session[session.action_type=='clickout item'].tail(1)['reference'].values[0]
clickout_itemDF = pd.DataFrame({'item_id':[clickout_item_id], 'clickout':[1]})
session_exploded = session_exploded.merge(clickout_itemDF, on='item_id', how='left').fillna(0)
session_exploded.head(7)

,item_id,price,rank,price_rank,clickout
0,55109,162,1,8,0.0
1,129343,25,2,1,0.0
2,54824,150,3,15,0.0
3,2297972,143,4,5,0.0
4,109014,101,5,12,0.0
5,1257342,49,6,16,1.0
6,1031578,118,7,20,0.0


#### session_duration

In [27]:
SessionDuration = session.timestamp.max() - session.timestamp.min()
session_exploded['session_duration'] = SessionDuration
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration
0,55109,162,1,8,0.0,0.0,1025
1,129343,25,2,1,0.0,0.0,1025


#### item_duration

In [22]:
ItemDuration = session.groupby('reference').timestamp.max() - session.groupby('reference').timestamp.min()
ItemDuration = ItemDuration.to_frame().reset_index().rename(columns={'reference':'item_id', 'timestamp':'item_duration'})
ItemDuration.head(2)

,item_id,item_duration
0,109038,11
1,1257342,0


In [25]:
session_exploded = session_exploded.merge(ItemDuration, on='item_id', how='left').fillna(0)
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,Session_Duration,item_duration
0,55109,162,1,8,0.0,1025,0.0
1,129343,25,2,1,0.0,1025,0.0


#### item_session_duration

In [29]:
ItemSessionDuration = session_exploded.item_duration/session_exploded.session_duration
session_exploded['item_session_duration'] = ItemSessionDuration
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration
0,55109,162,1,8,0.0,0.0,1025,0.0
1,129343,25,2,1,0.0,0.0,1025,0.0


#### item_interactions

In [42]:
ItemInteractions = session.groupby('reference').step.count()
ItemInteractions = ItemInteractions.to_frame().reset_index().rename(columns={'reference':'item_id', 'step':'item_interactions'})
session_exploded = session_exploded.merge(ItemInteractions, on='item_id', how='left').fillna(0)
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration,item_interactions
0,55109,162,1,8,0.0,0.0,1025,0.0,0.0
1,129343,25,2,1,0.0,0.0,1025,0.0,0.0


#### maximum_step

In [47]:
session_exploded['maximum_step'] = session.step.max()
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration,item_interactions,maximum_step
0,55109,162,1,8,0.0,0.0,1025,0.0,0.0,16
1,129343,25,2,1,0.0,0.0,1025,0.0,0.0,16


In [48]:
session_exploded.head()

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration,item_interactions,maximum_step
0,55109,162,1,8,0.0,0.0,1025,0.0,0.0,16
1,129343,25,2,1,0.0,0.0,1025,0.0,0.0,16
2,54824,150,3,15,0.0,0.0,1025,0.0,0.0,16
3,2297972,143,4,5,0.0,0.0,1025,0.0,0.0,16
4,109014,101,5,12,0.0,0.0,1025,0.0,0.0,16


#### top_list

In [60]:
top = [1, 1, 1, 1, 1]
session_exploded['top_list'] = pd.Series(top)
session_exploded.top_list = session_exploded.top_list.fillna(0)
session_exploded.head(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration,item_interactions,maximum_step,top_list
0,55109,162,1,8,0.0,0.0,1025,0.0,0.0,16,1.0
1,129343,25,2,1,0.0,0.0,1025,0.0,0.0,16,1.0


In [61]:
session_exploded.tail(2)

,item_id,price,rank,price_rank,clickout,item_duration,session_duration,item_session_duration,item_interactions,maximum_step,top_list
23,666936,112,24,22,0.0,0.0,1025,0.0,0.0,16,0.0
24,55088,118,25,17,0.0,0.0,1025,0.0,0.0,16,0.0


### Defined Function

In [0]:
trainlet[trainlet.action_type=='clickout item']

In [59]:
trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).impressions\
            .apply(lambda x: x.split('|')).explode().to_frame().reset_index().drop(columns='index')

,impressions
0,55109
1,129343
2,54824
3,2297972
4,109014
...,...
12028,747401
12029,239591
12030,2788070
12031,129853


In [60]:
trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).prices\
            .apply(lambda x: x.split('|')).explode().to_frame().reset_index().drop(columns='index')

,prices
0,162
1,25
2,150
3,143
4,101
...,...
12028,101
12029,107
12030,76
12031,140


In [58]:
trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).prices.apply(lambda x: x.split('|'))\
.apply(lambda x: list(range(1, len(x) + 1))).explode().to_frame().reset_index().drop(columns='index')

,prices
0,1
1,2
2,3
3,4
4,5
...,...
12028,21
12029,22
12030,23
12031,24


In [48]:
train[train.action_type=='clickout item'].groupby(['session_id', 'user_id']).tail(1).impressions\
            .apply(lambda x: x.split('|')).explode().to_frame().reset_index()

,index,impressions
0,15,55109
1,15,129343
2,15,54824
3,15,2297972
4,15,109014
...,...,...
15373343,13019555,2712342
15373344,13019555,48497
15373345,13019555,11933
15373346,13019555,1714483


The next step would be decreasing the what's in the function so it would be easier in running.

In [0]:
def transform_data(data):

  session_gb = data.groupby('session_id', sort=False)
  count = 0

  for session_id in data['session_id'].unique():
    try:
      session = session_gb.get_group(session_id)
      session_exploded = session[session.action_type=='clickout item'].tail(1)[['impressions']]
      session_exploded.impressions = session_exploded.impressions.apply(lambda x: x.split('|'))
      session_exploded = session_exploded.explode('impressions')
      session_exploded = session_exploded.rename(columns={'impressions':'item_id'})

      session_explode = session[session.action_type=='clickout item'].tail(1)[['prices']]
      session_explode.prices = session_explode.prices.apply(lambda x: x.split('|'))
      session_explode = session_explode.explode('prices')
      session_explode = session_explode.rename(columns={'prices':'price'})
      session_explode.price = session_explode.price.apply(lambda x: int(x))

      session_exploded['rank'] = range(1, len(session_exploded) + 1)

      session_exploded['price'] = session_explode.price
      session_exploded['price_rank'] = np.argsort(session_exploded['price'])

      clickout_item_id = session[session.action_type=='clickout item'].tail(1)['reference'].values.tolist()[0]
      clickout_itemDF = pd.DataFrame({'item_id':[clickout_item_id], 'clickout':[1]})
      session_exploded = session_exploded.merge(clickout_itemDF, on='item_id', how='left').fillna(0)

      SessionDuration = session.timestamp.max() - session.timestamp.min()
      session_exploded['session_duration'] = SessionDuration

      ItemDuration = session.groupby('reference').timestamp.max() - session.groupby('reference').timestamp.min()
      ItemDuration = ItemDuration.to_frame().reset_index()\
                                .rename(columns={'reference':'item_id', 'timestamp':'item_duration'})
      session_exploded = session_exploded.merge(ItemDuration, on='item_id', how='left').fillna(0)

      ItemSessionDuration = session_exploded.item_duration/session_exploded.session_duration
      session_exploded['item_session_duration'] = ItemSessionDuration

      ItemInteractions = session.groupby('reference').step.count()
      ItemInteractions = ItemInteractions.to_frame().reset_index()\
                                          .rename(columns={'reference':'item_id', 'step':'item_interactions'})
      session_exploded = session_exploded.merge(ItemInteractions, on='item_id', how='left').fillna(0)

      session_exploded['maximum_step'] = session.step.max()

      top = [1, 1, 1, 1, 1]
      session_exploded['top_list'] = pd.Series(top)
      session_exploded.top_list = session_exploded.top_list.fillna(0)

      if count == 0:
        Data = session_exploded
      else:
        Data = pd.concat([Data, session_exploded])
      
      count = count + 1
    except:
      continue

  return Data

In [0]:
def transform_data(data):

  session_gb = data.groupby('session_id', sort=False)
  count = 0

  for session_id in data['session_id'].unique():
    try:
      session = session_gb.get_group(session_id)
      session_exploded = session[session.action_type=='clickout item'].tail(1)[['impressions']]
      session_exploded.impressions = session_exploded.impressions.apply(lambda x: x.split('|'))
      session_exploded = session_exploded.explode('impressions')
      session_exploded = session_exploded.rename(columns={'impressions':'item_id'})

      session_explode = session[session.action_type=='clickout item'].tail(1)[['prices']]
      session_explode.prices = session_explode.prices.apply(lambda x: x.split('|'))
      session_explode = session_explode.explode('prices')
      session_explode = session_explode.rename(columns={'prices':'price'})
      session_explode.price = session_explode.price.apply(lambda x: int(x))

      session_exploded['rank'] = range(1, len(session_exploded) + 1)

      session_exploded['price'] = session_explode.price
      session_exploded['price_rank'] = np.argsort(session_exploded['price'])

      clickout_item_id = session[session.action_type=='clickout item'].tail(1)['reference'].values.tolist()[0]
      clickout_itemDF = pd.DataFrame({'item_id':[clickout_item_id], 'clickout':[1]})
      session_exploded = session_exploded.merge(clickout_itemDF, on='item_id', how='left').fillna(0)

      SessionDuration = session.timestamp.max() - session.timestamp.min()
      session_exploded['session_duration'] = SessionDuration

      ItemDuration = session.groupby('reference').timestamp.max() - session.groupby('reference').timestamp.min()
      ItemDuration = ItemDuration.to_frame().reset_index()\
                                .rename(columns={'reference':'item_id', 'timestamp':'item_duration'})
      session_exploded = session_exploded.merge(ItemDuration, on='item_id', how='left').fillna(0)

      ItemSessionDuration = session_exploded.item_duration/session_exploded.session_duration
      session_exploded['item_session_duration'] = ItemSessionDuration

      ItemInteractions = session.groupby('reference').step.count()
      ItemInteractions = ItemInteractions.to_frame().reset_index()\
                                          .rename(columns={'reference':'item_id', 'step':'item_interactions'})
      session_exploded = session_exploded.merge(ItemInteractions, on='item_id', how='left').fillna(0)

      session_exploded['maximum_step'] = session.step.max()

      top = [1, 1, 1, 1, 1]
      session_exploded['top_list'] = pd.Series(top)
      session_exploded.top_list = session_exploded.top_list.fillna(0)

      if count == 0:
        Data = session_exploded
      else:
        Data = pd.concat([Data, session_exploded])
      
      count = count + 1
    except:
      continue

  return Data

In [41]:
trainlet = train.iloc[0:10000]
trainlet.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [35]:
t1 = datetime.now()
x = transform_data(trainlet)
t2 = datetime.now()
time = t2 - t1
time

datetime.timedelta(0, 22, 7652)

In [36]:
x

,item_id,rank,price,price_rank,clickout,session_duration,item_duration,item_session_duration,item_interactions,maximum_step,top_list
0,55109,1,162,8,0.0,1025,0.0,0.0,0.0,16,1.0
1,129343,2,25,1,0.0,1025,0.0,0.0,0.0,16,1.0
2,54824,3,150,15,0.0,1025,0.0,0.0,0.0,16,1.0
3,2297972,4,143,5,0.0,1025,0.0,0.0,0.0,16,1.0
4,109014,5,101,12,0.0,1025,0.0,0.0,0.0,16,1.0
...,...,...,...,...,...,...,...,...,...,...,...
20,747401,21,101,6,0.0,194,0.0,0.0,0.0,3,0.0
21,239591,22,107,15,0.0,194,0.0,0.0,0.0,3,0.0
22,2788070,23,76,13,0.0,194,0.0,0.0,0.0,3,0.0
23,129853,24,140,14,0.0,194,0.0,0.0,0.0,3,0.0


In [22]:
train.session_id.nunique(), trainlet.session_id.nunique()

(745755, 581)

## Data Transformation

In [0]:
t1 = datetime.now()
Data = transform_data(train)
t2 = datetime.now()
time = t2 - t1
time

In [0]:
Data.to_csv('TrainData.csv', index=False)